# 0 Imports

In [38]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn import linear_model
# from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, recall_score, matthews_corrcoef, mean_absolute_error
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from statsmodels.stats.outliers_influence import variance_inflation_factor

In [39]:
data = pd.read_csv("../dat/dips/DIPS_Data.csv", sep = ",", low_memory = False)


# 1 Data Cleaning

## 1.1 Predictors and target

In [40]:
data.head()

,code,okayt1,okayt2,teilt2,intnr,datum,s1,s2,s3,s4,...,alk30,litbiery,litweiny,litspiry,litalky,grambiry,gramweiy,gramspiy,gramalky,filter_$
0,33,1,1,1,5,9/6/1996,74,1,3,1,...,0,0,.2,0,.2,0,20,0,20,1
1,35,1,1,1,21,9/21/1996,74,1,3,1,...,0,5.6,.375,.0045,5.9795,224,37.5,1.8,263.3,1
2,37,2,1,1,16,9/17/1996,74,2,3,1,...,0,.0625,.05,.00703125,.11953125,2.5,5,2.8125,10.3125,0
3,51,1,1,1,5,9/9/1996,75,1,3,1,...,0,0,.3,.005625,.305625,0,30,2.25,32.25,1
4,56,1,1,1,10,5/30/1997,75,1,3,1,...,1,.75,2.2,.01125,2.96125,30,220,4.5,254.5,1


### Predictors

In [41]:
predictors = ["whi_ges", "soz_ges", "gke_ges", "lzh_ges",
              "ile_sum1", "ile_sum2", "woc_gesp", "woc_gese",
              "asi_ges", "bsq_ges", "das_ges", "scl_gsi",
              # "scl_pst", "scl_psdi"
              ]

predictors_y = ["whi_gesy", "soz_gesy", "gke_gesy", "lzh_gesy",
                "asi_gesy", "bsq_gesy", "das_gesy", "scl_gsiy",
                # "scl_psty"
                ]

predictor_names = ["Positive mental health",    #whi_ges
                   "Social support",            #soz_ges (SAVE)
                   "General self-efficacy",     #gke_ges SAVE
                   "Life satisfaction",         #lzh_ges SAVE
                   "Stress 1",                  #ile_sum1 SAVE (life events?)
                   "Stress 2",                  #ile_sum2 SAVE (daily hassles?)
                   "Problem-focused coping",    #woc_gesp SAVE
                   "Emotion-focused coping",    #woc_gese SAVE
                   "Anxiety sensitivity",       #asi_ges SAVE
                   "Fear of Bodily Sensations", #bsq_ges SAVE
                   "Dysfunctional attitudes",   #das_ges SAVE
                   "General Psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   # "General Psychopathology: Positive Symptom Total (PST)",    #scl_pst SAVE
                   # "General Psychopathology: Positive Symptom Distress Index (PSDI)"    #scl_psdi SAVE
                   ]

extra_preds = ["fqn_ges",
               # "fq_ag", "fq_s", "fq_bi",
               "bdi_ges", "bai_ges",
               # "bdi_bel", "bai_bel",
               "ifa_int", "ifa_ext"]

extra_preds_y = ["fqn_gesy",
                 # "fq_agy", "fq_sy", "fq_biy",
                 "bdi_gesy", "bai_gesy",
                 # "bdi_bely", "bai_bely",
                 "ifa_inty", "ifa_exty"]


extra_preds_names = ["Phobia total",                     #fqn_ges (SAVE)
                     # "Agoraphobia",                      #fq_ag
                     # "Social phobia",                    #fq_s
                     # "Blood/Injection phobia",           #fq_bi
                     "Beck Depression Inventory",        #bdi_ges (SAVE)
                     # "Dysfunctional beliefs (from BDI)", #bdi_bel (SAVE)
                     "Beck Anxiety Inventory",           #bai_ges (SAVE)
                     # "Dysfunctional beliefs (from BAI)", #bai_bel (SAVE)
                     "Internal focus of attention", #ifa_int
                     "External focus of attention" #ifa_ext
                     ]

demographics = ["alter", "bild", "screen", "bmi", "bmi_kat", "ses_kom"]

demographics_y = ["bmiy", "bmi_katy"]

### Target

In [42]:
# get all columns that start with "hpi"
hpi_cols = [col for col in data.columns if col.startswith('hpi')]

# create a new column with 1 if any rows have a value of 2 in at least one of the columns starting with "hpi" and 0 if values in all columns starting with "hpi" are all 0, otherwise NA
# def classify_row(row):
#     if (row == 2).any():
#         return 1
#     elif (row == 0).all():
#         return 0
#     else:
#         return np.nan
#
# data['hpi'] = data[hpi_cols].apply(classify_row, axis=1)

# column that counts the number of columns with a value of 0
data['hpi_0_count'] = data[hpi_cols].apply(lambda x: (x == 0).sum(), axis=1)
# describe the new column
data['hpi_0_count'].describe()

# new column that is 0 if "hpig" is 0 and NA otherwise
data['hpi_0'] = np.where(data['hpig'] == 0, 0, np.nan)
data['hpi_0'].describe()

data['hpi'] = data[hpi_cols].apply(lambda x: 1 if any(x == 2) else 0, axis = 1)

# count the number of rows with a value of 1 in the new column
data['hpi'].value_counts(normalize = True)

hpi
0    0.721717
1    0.278283
Name: proportion, dtype: float64

In [45]:
data = data[predictors + extra_preds + demographics + predictors_y + extra_preds_y + demographics_y + ["hpi"]]

,whi_ges,soz_ges,gke_ges,lzh_ges,ile_sum1,ile_sum2,woc_gesp,woc_gese,asi_ges,bsq_ges,...,das_gesy,scl_gsiy,fqn_gesy,bdi_gesy,bai_gesy,ifa_inty,ifa_exty,bmiy,bmi_katy,hpi
0,6,3.45454545454545,24,2.66666666666667,10,11,30,22,25,2.29411764705882,...,77,.388888888888889,10,8,1,2,10,28.5779606767261,3,0
1,3,4.18181818181818,28,4,6,3,34,12,16,1.58823529411765,...,136,.1,14,5,5,0,6,21.192742415229,2,0
2,,,,,,,,,,,...,108,.0888888888888889,6,1,2,0,6,19.4931773879142,2,0
3,5,3.71428571428571,30,3.25,2,5,44,32,14,2.88235294117647,...,128,.622222222222222,34,6,5,2,4,20.6851502395123,2,1
4,10,4.04545454545454,27,3,1,8,28,42,15,1.64705882352941,...,119,.833333333333333,8,6,10,8,16,22.5308641975309,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,1,4,26,4.08333333333333,4,1,25,23,21,2.11764705882353,...,109,.111111111111111,13,4,2,0,8,21.9671201814059,2,0
1534,1,4.13636363636364,27,3.66666666666667,6,4,32,30,3,1.05882352941176,...,89,.188888888888889,11,0,1,0,2,21.6216216216216,2,0
1535,2,3.54545454545455,30,3.41666666666667,4,6,25,31,14,2,...,126,.311111111111111,6,6,6,0,10,17.7154195011338,1,0
1536,0,4.72727272727273,24,3.81818181818182,4,5,32,29,17,2.35294117647059,...,92,.0888888888888889,5,0,1,0,8,24.8015873015873,2,0


## 1.2 Missing values

In [46]:
data = data.replace(" ", np.nan)

In [50]:
for col in data.columns:
    print(data[col].value_counts())

whi_ges
1     396
0     332
2     261
3     176
4      96
5      60
6      46
7      23
8      14
10     11
9       7
11      4
12      1
13      1
Name: count, dtype: int64
soz_ges
4.77272727272727    77
4.72727272727273    74
4.86363636363636    72
4.68181818181818    72
4.54545454545454    71
                    ..
3.8                  1
4.42857142857143     1
2.59090909090909     1
1.86363636363636     1
3.47619047619048     1
Name: count, Length: 91, dtype: int64
gke_ges
30    166
29    154
28    132
27    120
31    102
25    101
26    100
32     83
24     69
34     61
33     55
23     44
35     43
22     33
36     32
20     21
21     21
37     20
39     13
19     10
18      9
38      8
17      7
40      6
16      5
15      4
14      3
12      2
10      1
13      1
11      1
Name: count, dtype: int64
lzh_ges
3.83333333333333    111
4                    89
3.75                 89
3.5                  86
3.58333333333333     79
                   ... 
1.09090909090909      1
4.63636

In [51]:
data["ses_kom"] = data["ses_kom"].replace(99, np.nan)

## 1.3 Data types

In [52]:
for col in data.columns:
    print(col, data[col].dtype)

whi_ges object
soz_ges object
gke_ges object
lzh_ges object
ile_sum1 object
ile_sum2 object
woc_gesp object
woc_gese object
asi_ges object
bsq_ges object
das_ges object
scl_gsi object
fqn_ges object
bdi_ges object
bai_ges object
ifa_int object
ifa_ext object
alter object
sex object
bild object
screen object
bmi object
bmi_kat object
ses_kom float64
whi_gesy object
soz_gesy object
gke_gesy object
lzh_gesy object
asi_gesy object
bsq_gesy object
das_gesy object
scl_gsiy object
fqn_gesy object
bdi_gesy object
bai_gesy object
ifa_inty object
ifa_exty object
bmiy object
bmi_katy object
hpi int64


In [53]:
# convert all columns to float
data = data.astype(float)

for col in data.columns:
    print(col, data[col].dtype)

whi_ges float64
soz_ges float64
gke_ges float64
lzh_ges float64
ile_sum1 float64
ile_sum2 float64
woc_gesp float64
woc_gese float64
asi_ges float64
bsq_ges float64
das_ges float64
scl_gsi float64
fqn_ges float64
bdi_ges float64
bai_ges float64
ifa_int float64
ifa_ext float64
alter float64
sex float64
bild float64
screen float64
bmi float64
bmi_kat float64
ses_kom float64
whi_gesy float64
soz_gesy float64
gke_gesy float64
lzh_gesy float64
asi_gesy float64
bsq_gesy float64
das_gesy float64
scl_gsiy float64
fqn_gesy float64
bdi_gesy float64
bai_gesy float64
ifa_inty float64
ifa_exty float64
bmiy float64
bmi_katy float64
hpi float64


## 1.4 Save data

In [54]:
data.to_csv("../dat/dips/DIPS_Data_cleaned.csv", sep = ",", index = False)

# 2 Data Exploration